In [0]:
import numpy as np
import pandas as pd


df=pd.read_csv("train.csv")



In [0]:
df=pd.read_csv("train.csv")

In [0]:
df.head(5)

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,Minor_Damage_And_Injuries,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352,7570
1,Minor_Damage_And_Injuries,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350,12128
2,Significant_Damage_And_Fatalities,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364,2181
3,Significant_Damage_And_Serious_Injuries,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728,5946
4,Significant_Damage_And_Fatalities,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883,9054


In [0]:
x=df.iloc[:,1:11].values

x

array([[4.92237443e+01, 1.40000000e+01, 2.20000000e+01, ...,
        3.13354768e+04, 3.00000000e+00, 4.24352083e-01],
       [6.24657534e+01, 1.00000000e+01, 2.70000000e+01, ...,
        2.60247111e+04, 2.00000000e+00, 3.52349973e-01],
       [6.30593607e+01, 1.30000000e+01, 1.60000000e+01, ...,
        3.92690539e+04, 3.00000000e+00, 3.36390699e-03],
       ...,
       [2.78538813e+01, 1.70000000e+01, 1.00000000e+00, ...,
        3.37719963e+04, 3.00000000e+00, 1.68173043e-01],
       [5.62100457e+01, 8.00000000e+00, 0.00000000e+00, ...,
        3.98204193e+04, 2.00000000e+00, 2.63809289e-02],
       [5.00000000e+01, 1.30000000e+01, 3.00000000e+00, ...,
        2.60053230e+04, 2.00000000e+00, 1.70358910e-02]])

In [0]:

y=df.iloc[:,0].values
y


array(['Minor_Damage_And_Injuries', 'Minor_Damage_And_Injuries',
       'Significant_Damage_And_Fatalities', ...,
       'Significant_Damage_And_Serious_Injuries',
       'Minor_Damage_And_Injuries', 'Highly_Fatal_And_Damaging'],
      dtype=object)

In [0]:
non_categorial_features = ['Minor_Damage_And_Injuries',
                          'Significant_Damage_And_Fatalities',
                          'Significant_Damage_And_Serious_Injuries',
                          'Highly_Fatal_And_Damaging']
                        

for categorical_feature in list(y):
    if categorical_feature not in non_categorial_features:
        y[categorical_feature] = y[categorical_feature].astype('category')
y_dummies=pd.get_dummies(y, sparse=True)

for i, col in enumerate(y_dummies.columns.tolist(), 1):
    y_dummies.loc[:, col] *= i
y_dummie = y_dummies.sum(axis=1)
y_dummie


0       2
1       2
2       3
3       4
4       3
       ..
9995    4
9996    1
9997    4
9998    2
9999    1
Length: 10000, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y_dummie,test_size=0.20)

In [0]:
from xgboost import XGBClassifier

In [0]:
classifier= XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.7,
              colsample_bynode=1, colsample_bytree=1, gamma=0.155
          , max_delta_step=0, max_depth=6,learning_rate=0.165,
              min_child_weight=1, missing=None, n_estimators=700, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
classifier.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.7,
              colsample_bynode=1, colsample_bytree=1, gamma=0.155,
              learning_rate=0.165, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=700, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
from sklearn.model_selection import GridSearchCV
parameters= [
  }]

grid_search = GridSearchCV(estimator=classifier,
                         param_grid= parameters,
                         scoring = 'accuracy',
                         cv=10,
                         n_jobs=-1)
grid_search= grid_search.fit(x_train,y_train) 
  

In [0]:
best_parameter = grid_search.best_params_
best_parameter

{'gamma': 0.155, 'n_estimators': 700}

In [0]:
best= grid_search.best_score_
best

0.9581249999999999

In [0]:
y_pred=classifier.predict(x_test)
y_pred


array([1, 1, 1, ..., 1, 4, 1])

In [0]:
y_test

4447    1
538     1
3380    1
7220    1
1585    3
       ..
80      2
5267    1
4281    1
8411    4
4192    1
Length: 2000, dtype: int64

In [0]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm


array([[562,  10,  13,   9],
       [  4, 479,   0,   4],
       [  3,   2, 362,   2],
       [  9,  14,   1, 526]])

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9645

In [0]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred, average='macro')


0.9652088086269079

In [0]:
from sklearn.model_selection import cross_val_score
accuracies= cross_val_score(estimator=classifier,X=x_train, y=y_train,cv=10)
accuracies.mean()
accuracies.std()

0.007871030745715575

In [0]:
accuracies.mean()


0.9581249999999999

In [0]:
test= pd.read_csv("test.csv")

In [0]:
test.head(5)

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,19.497717,16,6,72.151322,0.388959,78.32,4,37949.724386,2,0.069692,1
1,58.173516,15,3,64.585232,0.250841,78.60,7,30194.805567,2,0.002777,10
2,33.287671,15,3,64.721969,0.336669,86.96,6,17572.925484,1,0.004316,14
3,3.287671,21,5,66.362808,0.421775,80.86,3,40209.186341,2,0.199990,17
4,10.867580,18,2,56.107566,0.313228,79.22,2,35495.525408,2,0.483696,21


In [0]:
tst=test.iloc[:,0:10].values
tst

array([[1.94977169e+01, 1.60000000e+01, 6.00000000e+00, ...,
        3.79497244e+04, 2.00000000e+00, 6.96924473e-02],
       [5.81735160e+01, 1.50000000e+01, 3.00000000e+00, ...,
        3.01948056e+04, 2.00000000e+00, 2.77740022e-03],
       [3.32876712e+01, 1.50000000e+01, 3.00000000e+00, ...,
        1.75729255e+04, 1.00000000e+00, 4.31639490e-03],
       ...,
       [3.67123288e+01, 1.40000000e+01, 1.00000000e+01, ...,
        3.66761006e+04, 2.00000000e+00, 8.62895390e-03],
       [3.87214612e+01, 1.30000000e+01, 1.40000000e+01, ...,
        2.42113595e+04, 1.00000000e+00, 6.24651726e-03],
       [3.60730594e+01, 1.10000000e+01, 1.00000000e+00, ...,
        3.59201462e+04, 0.00000000e+00, 2.44090469e-02]])

In [0]:
y_pred1=classifier.predict(tst)
y_pred1.shape

(2500,)

In [0]:
def category(c):
  if c==1:
    return "Highly_Fatal_And_Damaging"
  elif c==2:
      return "Minor_Damage_And_Injuries"
  elif c==3:
        return "Significant_Damage_And_Fatalities"
  else:
          return "Significant_Damage_And_Serious_Injuries"
    
   
        

In [0]:
s= map(category,y_pred1)

In [0]:
w=list(s)

In [0]:
submission = pd.DataFrame({ 'Accident_ID': test["Accident_ID"],
                           'Severity':w })
submission.to_csv("my_submission4.csv", index=False)